In [1]:
import os

In [ ]:
# current working directory
import os
print(os.getcwd()) 

In [3]:
# change directory (parent directory)
os.chdir("../")     

In [ ]:
import os
print(os.getcwd())

In [48]:
from src.mlops_project.logger import logging
from pathlib import Path
from dataclasses import dataclass 

In [ ]:
@dataclass(frozen=True)
class DataIngestionConfig:
    """
    Data class for storing data ingestion config.
    """
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [80]:
from src.mlops_project.constants import *
from src.mlops_project.utils.common import read_yaml, create_directories
from box import ConfigBox

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        """
        Initialize ConfigurationManager with file paths for
        config, params and schema.
        Read YAML files and store them in ConfigBox objects.
        Create root directory for artifacts.
        """
        
        # Read the configuration, parameters, and schema files        
        self.config = ConfigBox(read_yaml(config_filepath))
        self.params = ConfigBox(read_yaml(params_filepath))
        self.schema = ConfigBox(read_yaml(schema_filepath))
        
        # Create the root directory for artifacts
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """
        Retrieve data ingestion configuration from main
        configuration.
        Create directories specified in the data
        ingestion configuration.
        Construct and return DataIngestionConfig object.
        """
        
        # Retrieve data ingestion configuration
        config = self.config.data_ingestion

        # Create root directory for data ingestion
        create_directories([config.root_dir])

        # Construct DataIngestionConfig object
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        # Log data ingestion config, return DataIngestionConfig object
        logging.info(f"Data Ingestion config: {data_ingestion_config}")
        return data_ingestion_config

In [94]:
import urllib.request as request
import zipfile

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        """
        Download file from source url
        Returns: None
        """
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logging.info(f"{filename} downloaded with following info: \n{headers}")
        else:
            logging.info(f"File already exists: {self.config.local_data_file}")

    def extract_zip_file(self):
        """
        Extracts zip file into data directory 
        Returns: None
        """  
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)


In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

# Defines a ConfigurationManager class that handles data
# ingestion config.
# __init__ method initializes ConfigurationManager with file paths
# for config, params and schema, reads the YAML files, stores them
# in ConfigBox objects, and creates the root directory for
# artifacts.
# Get_data_ingestion_config method retrieves data ingestion config
# from main config,
# creates directories specified in data ingestion config,
# constructs DataIngestionConfig object,
# logs data ingestion config, returns DataIngestionConfig object.